<a href="https://colab.research.google.com/github/ProgramerMetalhead/ABC-Monkey/blob/main/notebook_traning_unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT     = "/content/drive/MyDrive/Dataset Extendido 1"
IMG_DIR  = f"{ROOT}/data/images"
MASK_DIR = f"{ROOT}/data/masks"
OUT_WEIGHTS = f"{ROOT}/unet_best.weights.h5"

In [ ]:
IMG_SIZE  = 224   # igual que `AppConfig.TARGET_SIZE`
BATCH     = 8
EPOCHS    = 25

In [ ]:
!pip install opencv-python-headless==4.10.*
!pip install -q tqdm

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def mini_unet(input_shape=(224,224,3)):
    def dbl_conv(x,f):
        x = layers.Conv2D(f,3,padding='same',activation='relu')(x)
        x = layers.Conv2D(f,3,padding='same',activation='relu')(x)
        return x
    inputs = layers.Input(shape=input_shape)
    c1 = dbl_conv(inputs,16); p1 = layers.MaxPool2D()(c1)
    c2 = dbl_conv(p1,32);     p2 = layers.MaxPool2D()(c2)
    c3 = dbl_conv(p2,64)
    u4 = layers.UpSampling2D()(c3); u4 = layers.Concatenate()([u4,c2]); c4 = dbl_conv(u4,32)
    u5 = layers.UpSampling2D()(c4); u5 = layers.Concatenate()([u5,c1]); c5 = dbl_conv(u5,16)
    out = layers.Conv2D(1,1,activation='sigmoid')(c5)
    return models.Model(inputs,out,name="mini_unet")

model = mini_unet((IMG_SIZE,IMG_SIZE,3))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "mini_unet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 224, 224,  │      2,320 │ conv2d[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │      9,248 │ conv2d_2[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 56, 56,    │     36,928 │ conv2d_4[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 112, 112,  │          0 │ conv2d_5[0][0]    │
│ (UpSampling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 112, 112,  │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 96)               │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 112, 112,  │     27,680 │ concatenate[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 112, 112,  │      9,248 │ conv2d_6[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 224, 224,  │          0 │ conv2d_7[0][0]    │
│ (UpSampling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 224, 224,  │          0 │ up_sampling2d_1[… │
│ (Concatenate)       │ 48)               │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 224, 224,  │      6,928 │ concatenate_1[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 224, 224,  │      2,320 │ conv2d_8[0][0]  

 Total params: 118,273 (462.00 KB)

 Trainable params: 118,273 (462.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
import os
from pathlib import Path

def load_pair(img_path):
    fname   = tf.strings.split(img_path, os.sep)[-1]
    mask_path = tf.strings.join([MASK_DIR, fname], separator=os.sep)

    img  = tf.io.read_file(img_path)
    img  = tf.io.decode_image(img, channels=3, expand_animations=False)
    img  = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img  = tf.cast(img, tf.float32) / 255.0

    msk  = tf.io.read_file(mask_path)
    msk  = tf.io.decode_image(msk, channels=1, expand_animations=False)
    msk  = tf.image.resize(msk, (IMG_SIZE, IMG_SIZE), method="nearest")
    msk  = tf.cast(msk > 0, tf.float32)   # binaria 0/1
    return img, msk


In [ ]:
import tensorflow as tf

ds = file_ds.map(load_pair, num_parallel_calls=tf.data.AUTOTUNE).shuffle(1000)

# Convertir a int
ds_size = tf.data.experimental.cardinality(ds).numpy()
train_size = int(ds_size * 0.8)

train_ds = ds.take(train_size).batch(BATCH).prefetch(tf.data.AUTOTUNE)
val_ds   = ds.skip(train_size).batch(BATCH).prefetch(tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

ckpt = ModelCheckpoint(
    filepath=OUT_WEIGHTS,
    save_weights_only=True,
    save_best_only=False,     # <- guarda siempre
    save_freq='epoch',
    verbose=1)

es   = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True)

NameError: name 'OUT_WEIGHTS' is not defined

In [ ]:
from tqdm.keras import TqdmCallback

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[TqdmCallback(verbose=1)]   # barra con porcentaje
)

print(f"✅  Pesos guardados en: {OUT_WEIGHTS}")

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/25
848/848 ━━━━━━━━━━━━━━━━━━━━ 3050s 3s/step - accuracy: 0.9637 - loss: 0.1561 - val_accuracy: 0.9641 - val_loss: 0.1334
Epoch 2/25
848/848 ━━━━━━━━━━━━━━━━━━━━ 2567s 3s/step - accuracy: 0.9638 - loss: 0.1305 - val_accuracy: 0.9617 - val_loss: 0.1281
Epoch 3/25
848/848 ━━━━━━━━━━━━━━━━━━━━ 2625s 3s/step - accuracy: 0.9636 - loss: 0.1239 - val_accuracy: 0.9640 - val_loss: 0.0786
Epoch 4/25
848/848 ━━━━━━━━━━━━━━━━━━━━ 2577s 3s/step - accuracy: 0.9635 - loss: 0.0817 - val_accuracy: 0.9643 - val_loss: 0.0820
Epoch 5/25
848/848 ━━━━━━━━━━━━━━━━━━━━ 2502s 3s/step - accuracy: 0.9644 - loss: 0.0785 - val_accuracy: 0.9656 - val_loss: 0.0761
Epoch 6/25
848/848 ━━━━━━━━━━━━━━━━━━━━ 2520s 3s/step - accuracy: 0.9641 - loss: 0.0785 - val_accuracy: 0.9676 - val_loss: 0.0733
Epoch 7/25
848/848 ━━━━━━━━━━━━━━━━━━━━ 2476s 3s/step - accuracy: 0.9656 - loss: 0.0764 - val_accuracy: 0.9663 - val_loss: 0.0732
Epoch 8/25
848/848 ━━━━━━━━━━━━━━━━━━━━ 2491s 3s/step - accuracy: 0.9656 - loss: 0.0764 - 